In [166]:
with open("input.txt", "r") as f:
    inp = f.read()

inp_split = inp.split('\n\n')

rules = {k.split(': ')[0]: k.split(': ')[1] for k in inp_split[0].splitlines()}
mine = [int(i) for i in inp_split[1].splitlines()[1].split(',')]
others = [[int(x) for x in i.split(',')] for i in inp_split[2].splitlines()[1:]]

In [167]:
import re

def class_matches(rules, num):
    """ which class rules match a given number """
    found = {}
    for c, rule in rules.items():
        l1, h1, l2, h2 = [int(i) for i in re.split('[ or -]+', rule)]
        if num >= l1 and num <= h1:
            found[c] = 1
        elif num >= l2 and num <= h2:
            found[c] = 1
        else:
            found[c] = 0
    return found


""" Part 1 of the question, find the bad rate """
bad_rate = 0
good_tickets = []
for oth in others:
    good_one = True
    for num in oth:
        if sum(class_matches(rules, num).values()) == 0:
            bad_rate += num
            good_one = False
    if good_one:
        good_tickets.append(oth)

bad_rate


                

26026

In [168]:
# Part 2 is to take the good tickets only and find which rules apply

match = {}
for ticket in good_tickets:
    for i in range(len(ticket)):
        if not i in match:
            match[i] = class_matches(rules, ticket[i])
        else:
            vals = class_matches(rules, ticket[i])
            for key in list(rules.keys()):
                match[i][key] += vals[key]

In [169]:
import pandas as pd

df = pd.DataFrame(match)

num_good_tickets = len(good_tickets)

# Make it 1 if all rules can be applied and 0 if not
df_match = df.where(df == num_good_tickets, other=0).mask(df == num_good_tickets, 1)

In [170]:
# Sort this so the lower triagle contains all the 1s

includingtotals = pd.concat([df_match,pd.DataFrame(df_match.sum(axis=1),columns=['Total'])],axis=1)
sorted = includingtotals.sort_values(['Total'], ascending=[True])
sorted = sorted.drop(columns=['Total'], axis=1)
sorted = sorted.reindex(sorted.sum().sort_values(ascending=False).index, axis=1)
sorted

,5,12,13,11,16,0,8,1,19,10,3,6,9,14,7,4,17,2,18,15
arrival location,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
arrival platform,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
row,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
class,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
arrival station,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
zone,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
departure date,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
departure location,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
departure time,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
departure platform,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0


In [171]:
# From above, I see there is a pattern. The diagonal contains the only rules/class pairs.

rule_order = pd.Series(np.diag(sorted), index=[sorted.columns, sorted.index]).sort_index()

In [172]:
# Now get the depature indexes
dep_index = [i for i, si in enumerate(rule_order.index.values) if si[1].startswith('departure')]

In [173]:
# Take the indexes from my ticket
a = np.take(mine, dep_index)

In [174]:
# Answer is the product
np.product(a)

1305243193339